    Radial Distribution Function

Your homework is to calculate the radial distribution function for the final coordinates after 50,000 steps of MC simulation. You should pick another temperature to perform a second simulation and compare the two RDFs. A function for computing the RDF can be provided to you upon request, but it is a bonus if you write your own. You can see some instructions for writing calculating the RDF here. If you prefer to use the provided function, you can find it here.


In [ ]:
import math

import random

import time

start = time.time()




def calculate_total_energy(coordinates, box_length, cutoff):
    """
    Calculate the total energy of a set of particles using the Lennard Jones potential.
    
    Parameters
    ----------
    coordinates : list
        A nested list containing the x, y,z coordinate for each particle
    box_length : float
        The length of the box. Assumes cubic box.
    cutoff : float
        The cutoff length
    
    Returns
    -------
    total_energy : float
        The total energy of the set of coordinates.
    """
    
    total_energy = 0
    num_atoms = len(coordinates)

    for i in range(num_atoms):
        for j in range(i+1, num_atoms):
            # Calculate the distance between the particles - exercise.
            dist_ij = calculate_distance(coordinates[i], coordinates[j], box_length)

            if dist_ij < cutoff:
                # Calculate the pairwise LJ energy
                LJ_ij = calculate_LJ(dist_ij)

                # Add to total energy.
                total_energy += LJ_ij
    return total_energy

def read_xyz(filepath):
    """
    Reads coordinates from an xyz file.
    
    Parameters
    ----------
    filepath : str
       The path to the xyz file to be processed.
       
    Returns
    -------
    atomic_coordinates : list
        A two dimensional list containing atomic coordinates
    """
    
    with open(filepath) as f:
        box_length = float(f.readline().split()[0])
        num_atoms = float(f.readline())
        coordinates = f.readlines()
    
    atomic_coordinates = []
    
    for atom in coordinates:
        split_atoms = atom.split()
        
        float_coords = []
        
        # We split this way to get rid of the atom label.
        for coord in split_atoms[1:]:
            float_coords.append(float(coord))
            
        atomic_coordinates.append(float_coords)
        
    return atomic_coordinates, box_length

def calculate_LJ(r_ij):
    """
    The LJ interaction energy between two particles.

    Computes the pairwise Lennard Jones interaction energy based on the separation distance in reduced units.

    Parameters
    ----------
    r_ij : float
        The distance between the particles in reduced units.
    
    Returns
    -------
    pairwise_energy : float
        The pairwise Lennard Jones interaction energy in reduced units.

    Examples
    --------
    >>> calculate_LJ(1)
    0

    """
    
    r6_term = math.pow(1/r_ij, 6)
    r12_term = math.pow(r6_term, 2)
    
    pairwise_energy = 4 * (r12_term - r6_term)
    
    return pairwise_energy


## Add your group's modified calculate_distance function
def calculate_distance(coord1, coord2, box_length=None):
    """
    Calculate the distance between two points. When `box_length` is set, the minimum image convention is used to calculate the distance between the points.

    Parameters
    ----------
    coord1, coord2 : list
        The coordinates of the points, [x, y, z]
    
    box_length : float, optional
        The box length. This function assumes box is a cube.

    Returns
    -------
    distance : float
        The distance between the two points.
    """
    
    distance = 0
    
    for i in range(3):
        dim_dist = (coord1[i] - coord2[i])
        if box_length:
            dim_dist = (dim_dist - box_length * round(dim_dist/ box_length))**2
        distance += dim_dist
        
        
    distance = math.sqrt(distance)
    
    return distance


## Add your group's tail correction function
import os

config1_file = os.path.join("../../lj_sample_configurations", "lj_sample_config_periodic1.txt")

sample_coords, box_length = read_xyz(config1_file)

print(box_length)

print(len(sample_coords))

def calculate_tail_correction(n,b,r_c):
    r3_term = math.pow(1/r_c, 3)
    r9_term = (1/3) * (math.pow(r3_term, 9))
    NV_term = (8*math.pi/3) * ((n**2)/(b**3))
    tail_correction = NV_term * (r9_term - r3_term)
    
    return tail_correction

In [2]:
def accept_or_reject(delta_U, beta):
    """
    
    
    
    
    
    Accept or reject a move based on metropolis criterion.
    
    Paramaters
    ----------------
    delta_U : float
    
        change in energy for moving systems from m to n
    
    
    
    beta : float
        1/temperature
    
    
    
    Returns
    --------
    bool
        whether move is accepted-true
        
    
        
    
    
    
    
    
    """
    
    
    if delta_U <= 0.0:
        accept = True
    else:
        #Gen random number on(0,1)
        random_number = random.random()
        p_acc = math.exp(-beta*delta_U)
        
        if random_number < p_acc:
            accept = True
         
        else:
            accept = False
            
    return accept

In [ ]:
def calculate_pair_energy(coordinates, i_particle, box_length, cutoff):
    
    """
    
    Calculate interaction eenrgy of particle w/ its environment (all other particles in sys)
    
    Parameters
    ----------------
    coordinates : list
        the coordinates for all particles in sys
    i_particle : int
        particle number for which to calculate energy
        
    cutoff : float
        simulation cutoff. beyond distnaces, interactions aren't calculated
    
    box length : float
    
        length of simultion box. assumse cubic boc
        
    
    Returns
    ---------------
    
    float
        pairwise interaction energy of ith particle w/all other particles in sys
        
    
    
    
    
    
    """
    
    e_total = 0.0
    i_position = coordinates[i_particle]
    
    num_atoms = len(coordinates)
    
    for j_particle in range(num_atoms):
        if i_particle != j_particle:
            
            j_position = coordinates[j_particle]
            rij = calculate_distance(i_position, j_position, box_length)
            
            if rij < cutoff:
                e_pair = calculate_LJ(rij)
                e_total += e_pair
                
                

    return e_total

In [ ]:
#Read or generate inital coordiantes


coordinates, box_length = read_xyz('../../lj_sample_configurations/lj_sample_config_periodic1.txt')
# Set simulation parameters
reduced_temperature = 0.9

num_steps = 5000

max_displacement = 0.1

cutoff = 3

freq = 1000

# Calculated quantities

beta = 1 / reduced_temperature
num_particles = len(coordinates)

# Energy calculations 

total_energy = calculate_total_energy(coordinates, box_length, cutoff)
print(total_energy)

tail_correction = calculate_tail_correction(num_particles, box_length, cutoff)

print(tail_correction)

total_energy += tail_correction

for step in range(num_steps):
        
        # 1. Randomly pick one of particles
        random_particle = random.randrange(num_particles)
        
        # 2. Calculate in interavtion energy of sleectd particle w/system and store this value
        current_energy = calculate_pair_energy(coordinates, random_particle, box_length, cutoff)
       
        # 3. Gnerate random x,y,z displacement
        
        x_rand = random.uniform(-max_displacement, max_displacement)
        y_rand = random.uniform(-max_displacement, max_displacement)
        z_rand = random.uniform(-max_displacement, max_displacement)


        #4. Modify coordinate of Nth particle by genrated displacements
        coordinates[random_particle][0] += x_rand
        coordinates[random_particle][1] += y_rand
        coordinates[random_particle][2] += z_rand
        
        #5. Calculate interaction energy of moved particle w/system
        proposed_energy = calculate_pair_energy(coordinates, random_particle, box_length, cutoff)
        delta_energy = proposed_energy - current_energy
        #6. Calculate if we accept move based on energy diff
        
        accept = accept_or_reject(delta_energy, beta)
        #7. if accept, move particle
        if accept: 
            total_energy += delta_energy
        else:
            # Move is not accepted, roll back coordinates
            coordinates[random_particle][0] -= x_rand
            coordinates[random_particle][1] -= y_rand
            coordinates[random_particle][2] -= z_rand


        #8. print energy if step is multiple of freq
        
        if step % freq == 0:
            print(step, total_energy/num_particles)

            
            
            

How to calculate the pair correlation function g(r)

This explanation is for three-dimensional data. To calculate g(r), do the following:

    Pick a value of dr
    Loop over all values of r that you care about:
        Consider each particle you have in turn. Count all particles that are a distance between r and r + dr away from the particle you're considering. You can think of this as all particles in a spherical shell surrounding the reference particle. The shell has a thickness dr.
        Divide your total count by N, the number of reference particles you considered -- probably the total number of particles in your data.
        Divide this number by 4 pi r^2 dr, the volume of the spherical shell (the surface area 4 pi r^2, multiplied by the small thickness dr). This accounts for the fact that as r gets larger, for trivial reasons you find more particles with the given separation.
        Divide this by the particle number density. This ensures that g(r)=1 for data with no structure. In other words, if you just had an arbitrarily placed spherical shell of inner radius r and outer radius r+dr, you'd expect to find about rho * V particles inside, where rho is the number density and V is the volume of that shell. 
    In 2D, follow the algorithm as above but divide by 2 pi r dr instead of step #3 above. 

One caveat: For experimental data, you often have edges to your sample that are artificial. For example, you take a picture of particles but at the edges of your picture, the system extends further outwards. Thus when calculating g(r) based on reference particles near the edge of your image, you have a problem. You'll have to modify step #3 above with the correct volume/area that actually lies within the image you're looking at. The routines I've written take care of that.

I wrote IDL routines to calculate g(r) in 2D and 3D. These do some special tricks to deal with experimental data, in other words, to cleverly deal with the edge effects.

2D program: For a particle near the edge of a rectangular image, when I'm counting particles a distance r away from it, for many values of r the circle of radius r extends outside of the image. I did some math to figure out how to determine how much angular extent of the circle lies within the image for these cases (in the subroutine checkquadrant). Thus the edges are correctly accounted for.

3D program: For a particle near the edge of a rectangular image box, when I'm counting particles a distance r away from it, we have the same problem described for 2D data. However, calculating the resulting solid angle of the sphere contained within the box is more than I could handle mathematically, for arbitrary box dimensions and arbitrary locations within the box. So I did a trick. My image boxes tend to be short and wide, that is, very narrow in Z and large in X and Y.

My program does the following. I'm calculating g(r) for r < rmax, with a default rmax = 10. I consider only reference particles that are more than rmax away from the horizontal edges of the box, so I never have to worry about overlaps in X and Y. The resulting formula, to worry about overlaps in Z alone, turns out to be quite reasonable. (That is, I calculate the surface area of a spherical hemisphere of radius R that is cut off at a finite height H < R. It turns out this formula is simple: 2 pi R H.)

So, an important warning: Don't choose rmax to be more than half of the width of your data, in X and Y. This restriction is only for the 3D program.

See here for additional comments on my "special tricks".



In [3]:
import matplotlib.pyplot as plt 
import scipy as sp
 
def distance(a, b):
    dx = abs(a[0] - b[0])
    x = min(dx, abs(A - dx))
     
    dy = abs(a[1] - b[1])
    y = min(dy, abs(B - dy))
     
    dz = abs(a[2] - b[2])
    z = min(dz, abs(C - dz))
 
    return sp.sqrt(x**2 + y**2 + z**2)


def volume(z, r, z_bot, z_top):
    """ volume of a sphere of radius r located at height z """
    volume = 4.0 / 3.0 * sp.pi * r**3
     
    """ cut off a spherical cap if the sphere extends below z_bot """
    if z - r < z_bot:
        h = z_bot - (z - r)
        volume -= sp.pi * h**2 * (r - h / 3.0)
 
    """ cut off a spherical cap if the sphere extends above z_top """
    if z + r > z_top:
        h = z + r - z_top
        volume -= sp.pi * h**2 * (r - h / 3.0)
     
    return volume

class Trajectory:
    def __init__(self, filename, skip, z_bot_interface, z_top_interface,
                 interface_offset=4.0, resolution=200):
 
        self.filename = filename
        self.skip = skip
        self.z_top = z_top_interface - interface_offset
        self.z_bot = z_bot_interface + interface_offset
        self.resolution = resolution
         
        self.parse_input()
        self.compute_volume_per_h2o()
 
    ...
    
    
class Trajectory:
    ...
    def parse_input(self):
        with open(self.filename, 'r') as f:
            data = f.readlines()
 
        self.n_atoms = int(data[0].split()[0])
        self.n_steps_total = int(len(data) / (self.n_atoms + 2))
        self.n_steps = self.n_steps_total // self.skip
         
        self.atom_list = [line.split()[0] for line in
                          data[2 : self.n_atoms + 2]]
 
        self.coordinates = sp.zeros((self.n_steps, self.n_atoms, 3))
        for step in range(self.n_steps):
            coords = sp.zeros((self.n_atoms, 3))
             
            i = step * self.skip * (self.n_atoms + 2)
            for j, line in enumerate(data[i + 2 : i + self.n_atoms + 2]):
                coords[j, :] = [float(value) for value in line.split()[1:]]
             
            self.coordinates[step] = coords
 
    ...
    
    
class Trajectory:
    ...
     
    def compute_volume_per_h2o(self):
        n_h2o = 0
        for step in range(self.n_steps):
            for i, atom in enumerate(self.coordinates[step]):
                z = atom[2]
                if self.atom_list[i] == "O" and self.z_bot < z < self.z_top:
                    n_h2o += 1
         
        volume = A * B * (self.z_top - self.z_bot)
        average_n_h2o = n_h2o / self.n_steps
        self.volume_per_h2o = volume / average_n_h2o
 
    ...
    
    

class Trajectory:
    ...
     
    def compute_radial_distribution(self):
        """ no reason to go above half of the smallest lattice parameter
            as mirror images start to be double-counted """
        r_cutoff = min(A, B) / 2.0
 
        dr = r_cutoff / self.resolution
        self.radii = sp.linspace(0.0, self.resolution * dr, self.resolution)
 
        volumes = sp.zeros(self.resolution)
        self.g_of_r = sp.zeros(self.resolution)
         
        for step in range(self.n_steps):
            print('{:4d} : {:4d}'.format(step, self.n_steps))
             
            """ isolate all liquid water molecules based on their oxygens """
            data_oxygen = []
            for i, atom in enumerate(self.coordinates[step]):
                if self.atom_list[i] == "O":
                    if self.z_bot < atom[2] < self.z_top:
                        data_oxygen.append(atom)
            data_oxygen = sp.array(data_oxygen)
             
            for i, oxygen1 in enumerate(data_oxygen):
                """ calculate the volume of the cut spherical shells """
                for j in range(self.resolution):
                    r1 = j * dr
                    r2 = r1 + dr
                    v1 = volume(oxygen1[2], r1, self.z_bot, self.z_top)
                    v2 = volume(oxygen1[2], r2, self.z_bot, self.z_top)
                    volumes[j] += v2 - v1
 
                """ loop over pairs of O atoms, each pair counts as 2 """
                for oxygen2 in data_oxygen[i:]:
                    dist = distance(oxygen1, oxygen2)
                    index = int(dist / dr)
                    if 0 < index < self.resolution:
                        self.g_of_r[index] += 2.0
         
        for i, value in enumerate(self.g_of_r):
            self.g_of_r[i] = value * self.volume_per_h2o / volumes[i]
 
    ...
    

    
    
class Trajectory:
    ...
 
    def plot(self, filename=""):
        """ plots the radial distribution function
        if filename is specified, prints it to file as a png """
         
        if not self.g_of_r.any():
            print('compute the radial distribution function first\n')
            return
         
        plt.xlabel('r (Å)')
        plt.ylabel('g$_{OO}$(r)')
        plt.plot(self.radii, self.g_of_r)
         
        if filename:
            plt.savefig('radial_distribution_function.png', dpi=300,
                        bbox='tight', format='png')
         
        plt.show()
        
        
A = 12.991
B = 11.832
C = 30.577
bottom_interface = 23.0
top_interface = 40.0
 
TiO2_H2O = Trajectory('TiO2_H2O-pos-1.xyz', 10, 
                      bottom_interface, top_interface)
TiO2_H2O.compute_radial_distribution()
TiO2_H2O.plot()

ModuleNotFoundError: No module named 'scipy'

In [1]:
from pylj import md, sample

def md_simulation(number_of_particles, temperature, 
                  box_length, number_of_steps, 
                  sample_frequency):
    """
    Runs a molecular dynamics simulation in using the pylj 
    molecular dynamics engine.
    
    Parameters
    ----------
    number_of_particles: int
        The number of particles in the simulation
    temperature: float
        The temperature for the initialisation and 
        thermostating
    box_length: float
        The length of the simulation square
    number_of_steps: int
        The number of molecular dynamics steps to run
    sample_frequency: 
        How regularly the visualisation should be updated
        
    Returns
    -------
    pylj.util.System
        The complete system information from pylj
    """
    %matplotlib notebook
    system = md.initialise(number_of_particles, temperature, 
                           box_length, 'square')
    sample_system = sample.RDF(system)
    system.time = 0
    for i in range(0, number_of_steps):
        system.integrate(md.velocity_verlet)
        system.md_sample()
        system.heat_bath(temperature)
        system.time += system.timestep_length
        system.step += 1
        if system.step % sample_frequency == 0:
            sample_system.update(system)
    sample_system.average()
    return system

system = md_simulation(20, 300, 20, 2000, 25)

ModuleNotFoundError: No module named 'pylj'